In [1]:
import os
import json
import pandas as pd
import numpy as np

In [ ]:
datafile = os.path.join('/Users/enricozhou/Desktop/abc/dat/reddit/2018')
datafile = os.path.join('../../../dat/reddit/2018')

with open(datafile, 'r') as f:
    record_dicts = []
    for line in f.readlines():
        record = json.loads(line)
        reply_list = record['reply']
        earliest_reply_text = None
        for reply_dict in sorted(reply_list, key=lambda x: x['created_utc']):
            if reply_dict['body'] != '[deleted]':
                earliest_reply_text = reply_dict['body']
            if earliest_reply_text:
                break
        if earliest_reply_text:
            record.pop('reply')
            record['reply_text'] = earliest_reply_text
            record_dicts.append(record)

: 

In [ ]:
reddit_df = pd.DataFrame(record_dicts)
reddit_df = reddit_df[reddit_df.body != '[deleted]']
reddit_df = reddit_df.astype({'score':np.int64, 'controversiality':np.int64, 'gilded':np.int64, 'created_utc':np.int64})

: 

In [12]:
reddit_df.head()

,gender,body,author,author_flair_text,created_utc,link_id,score,controversiality,gilded,id,subreddit,author_flair_css_class,parent_id,reply_text
0,female,if this isn’t accurate then idk what is,be-concerned,15F,1542407736,t3_9xpaoc,60,0,0,e9ufsur,infj,sun,e9ufsur,"Yeah, this is definitely me. Then again, I cou..."
1,female,"i’m in highschool right now. two years ago, my...",be-concerned,15F,1545673571,t3_a940fq,5,0,0,ecgumma,infj,sun,ecgumma,Identify a common interest you have and maybe ...
2,female,[It’s me.](https://imgur.com/a/WDavpAM),INFJen,27F,1525914365,t3_8hz688,3,0,0,dyqcclj,infj,clover,dyqcclj,Aphrodite? Is that you?
3,female,u/User_Simulator u/INFJen,INFJen,27F,1528055310,t3_8o8shk,2,0,0,e01ztf1,infj,clover,e01ztf1,+/u/User_Simulator /u/INFJen
4,female,"It's okay, you too are unique. I'm unique, you...",INFJen,27F,1522538912,t3_88m25u,8,0,0,dwlp1jv,infj,clover,dwlp1jv,[Are we?](https://www.wikihow.com/Stop-Taking-...


In [13]:
gold = reddit_df[reddit_df.gilded>0]
contro = reddit_df[reddit_df.controversiality>0]

In [14]:
reddit_df['sub_id'] = pd.Categorical(reddit_df['subreddit']).codes
data = reddit_df[['body', 'score', 'sub_id']]
data = data.astype({'score':np.int64})
text = data['body'].values
labels = data['score'].values

In [15]:
data.head()

,body,score,sub_id
0,if this isn’t accurate then idk what is,60,12
1,"i’m in highschool right now. two years ago, my...",5,12
2,[It’s me.](https://imgur.com/a/WDavpAM),3,12
3,u/User_Simulator u/INFJen,2,12
4,"It's okay, you too are unique. I'm unique, you...",8,12


In [16]:
reddit_df.head()

,gender,body,author,author_flair_text,created_utc,link_id,score,controversiality,gilded,id,subreddit,author_flair_css_class,parent_id,reply_text,sub_id
0,female,if this isn’t accurate then idk what is,be-concerned,15F,1542407736,t3_9xpaoc,60,0,0,e9ufsur,infj,sun,e9ufsur,"Yeah, this is definitely me. Then again, I cou...",12
1,female,"i’m in highschool right now. two years ago, my...",be-concerned,15F,1545673571,t3_a940fq,5,0,0,ecgumma,infj,sun,ecgumma,Identify a common interest you have and maybe ...,12
2,female,[It’s me.](https://imgur.com/a/WDavpAM),INFJen,27F,1525914365,t3_8hz688,3,0,0,dyqcclj,infj,clover,dyqcclj,Aphrodite? Is that you?,12
3,female,u/User_Simulator u/INFJen,INFJen,27F,1528055310,t3_8o8shk,2,0,0,e01ztf1,infj,clover,e01ztf1,+/u/User_Simulator /u/INFJen,12
4,female,"It's okay, you too are unique. I'm unique, you...",INFJen,27F,1522538912,t3_88m25u,8,0,0,dwlp1jv,infj,clover,dwlp1jv,[Are we?](https://www.wikihow.com/Stop-Taking-...,12


In [17]:
reddit_df['sub_id'].head()

0    12
1    12
2    12
3    12
4    12
Name: sub_id, dtype: int8

In [18]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


True

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
# from nltk.corpus import stopwords
# stoplist = stopwords.words("English")

# Creiamo un oggetto CountVectorizer per trasformare il testo in una matrice di conteggi
vec = CountVectorizer(
    max_df=0.9,         # ignora le parole che compaiono in più del 90% dei documenti (troppo comuni, poco informative)
    min_df=0.02,        # ignora le parole che compaiono in meno del 2% dei documenti (troppo rare)
    stop_words='english', # rimuove automaticamente le stopwords inglesi (come 'the', 'and', 'is', ecc.)
    ngram_range=(1,2)   # considera sia unigrammi (singole parole) che bigrammi (sequenze di 2 parole) esempio: ['I', 'love', 'pizza', 'I love', 'love pizza']

)

# Trasforma il testo in una matrice di features (documenti x termini)
# text dovrebbe essere una lista di documenti (stringhe)
# creazione di una Bag of Words (BoW) con i filtri specificati
features = vec.fit_transform(text)

# Mostra la dimensione della matrice risultante:
# - features.shape[0] = numero di documenti
# - features.shape[1] = numero di termini selezionati dopo i filtri
# righe e colonne
features.shape


(422206, 195)

In [20]:
features.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
from sklearn.linear_model import LinearRegression, Lasso   # Importiamo i modelli di regressione lineare
from sklearn.metrics import mean_squared_error as mse      # Importiamo la funzione per calcolare il Mean Squared Error (MSE)

# Creiamo un array con gli indici di tutti i documenti
# features.shape[0] = numero di documenti nella matrice bag-of-words
docs = np.arange(features.shape[0])  

# Mescoliamo casualmente l'ordine dei documenti per evitare bias nell'addestramento
np.random.shuffle(docs)

# Qui usiamo tutti i documenti mescolati come "subsample"
# (Potremmo anche scegliere solo una parte es. docs[:500])
subsample = docs
# subsample = np.random.shuffle(docs)

# Riordiniamo le feature e le etichette (labels) in base al nuovo ordine casuale
tr_features = features[subsample, :]    # matrice bag-of-words/n-grammi dei documenti mescolati
tr_labels = labels[subsample]           # punteggi dei documenti (score) mescolati

# Stampiamo la forma della matrice (num_documenti, num_feature)
print(tr_features.shape)

# Creiamo un modello di regressione lineare
model = LinearRegression()

# Alleniamo il modello usando le feature (testo) per predire i punteggi
model.fit(tr_features, tr_labels)

# Usiamo il modello addestrato per predire i punteggi sugli stessi dati di training
train_pred = model.predict(tr_features)

# Calcoliamo l'errore quadratico medio (MSE) tra valori reali e predetti
print(mse(tr_labels, train_pred))

# Stampiamo i punteggi reali (etichette originali)
print(tr_labels)

# Stampiamo i punteggi predetti dal modello
print(train_pred)


(422206, 195)
27349.35425730105
[  2   2   1 ...   2   3 124]
[25.15380765  4.70403123 23.55221749 ... 29.47121705 15.9732831
 25.7932776 ]


In [22]:
# Documento corrisponde a ogni riga della matrice
print(docs)
print(subsample)
print(tr_features) # (0, 36)    2 ➡ Documento 0 contiene 2 occorrenze della parola numero 36 del vocabolario
print(tr_labels) # array con i punteggi reali (target) per ogni documento.

[312738 247679  66614 ... 275330  15138 333957]
[312738 247679  66614 ... 275330  15138 333957]
  (0, 4)	1
  (0, 26)	1
  (0, 29)	1
  (0, 93)	1
  (0, 98)	1
  (0, 119)	2
  (0, 141)	1
  (0, 152)	1
  (0, 162)	1
  (0, 170)	1
  (0, 171)	1
  (0, 177)	1
  (0, 183)	1
  (1, 3)	1
  (1, 60)	1
  (1, 61)	1
  (1, 63)	1
  (1, 99)	1
  (1, 107)	1
  (1, 118)	1
  (1, 179)	2
  (1, 194)	1
  (2, 93)	1
  (2, 110)	1
  (2, 167)	1
  :	:
  (422202, 47)	1
  (422202, 48)	1
  (422202, 63)	1
  (422202, 93)	1
  (422202, 148)	1
  (422203, 91)	1
  (422204, 19)	1
  (422204, 24)	1
  (422204, 31)	1
  (422204, 35)	1
  (422204, 36)	1
  (422204, 39)	1
  (422204, 40)	2
  (422204, 99)	1
  (422204, 110)	1
  (422204, 112)	1
  (422204, 123)	2
  (422204, 147)	1
  (422204, 159)	1
  (422204, 162)	1
  (422204, 166)	1
  (422204, 171)	1
  (422204, 176)	1
  (422204, 180)	1
  (422204, 188)	1
[  2   2   1 ...   2   3 124]


In [23]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error as mse

docs = np.arange(features.shape[0])
np.random.shuffle(docs)
subsample = docs
tr_features = features[subsample, :]
# tr_features = np.asarray(features[subsample, :])
# tr_features = np.asarray(tr_features.sum(axis=1)) # sommando lungo l’asse delle parole, ottieni UN SOLO NUMERO per documento, cioè la somma totale di parole (o n-grammi) in quel documento.
'''
Doc0 → 57 parole totali  
Doc1 → 123 parole totali  
Doc2 → 14 parole totali  
...
'''
tr_labels = labels[subsample]
print(tr_features.shape)

model = LinearRegression()
model.fit(tr_features, tr_labels)
train_pred = model.predict(tr_features)
print(mse(tr_labels, train_pred))

(422206, 195)
27349.35425730103


In [24]:
type(tr_features)
type(tr_labels)

numpy.ndarray

In [25]:
np.sqrt(mse(tr_labels, train_pred)) # calcola la radice quadrata dell'errore quadratico medio (RMSE)
# il modello in media sbaglia di circa 165 punti, che è enorme se i valori stanno tra 0 e 100 → significa che non sta imparando bene.

165.37640175460655

In [26]:
reddit_df.head(2)

,gender,body,author,author_flair_text,created_utc,link_id,score,controversiality,gilded,id,subreddit,author_flair_css_class,parent_id,reply_text,sub_id
0,female,if this isn’t accurate then idk what is,be-concerned,15F,1542407736,t3_9xpaoc,60,0,0,e9ufsur,infj,sun,e9ufsur,"Yeah, this is definitely me. Then again, I cou...",12
1,female,"i’m in highschool right now. two years ago, my...",be-concerned,15F,1545673571,t3_a940fq,5,0,0,ecgumma,infj,sun,ecgumma,Identify a common interest you have and maybe ...,12


In [27]:
# Un flair è un piccolo testo (o un’icona) associato all’autore di un post/commento.
'''
Serve come etichetta visiva, per esempio:
Età + Genere → 25M, 30/F, F/28
Ruolo → Moderator
Categoria → Expert, Beginner
Ogni subreddit può definire i propri flair liberi (inseriti dall’utente) o standardizzati (scelti da un menu).

Nei dati di Reddit, li trovi in due colonne:
author_flair_text → testo libero del flair (es. "28M", "F/29", "Mod")
author_flair_css_class → classe CSS usata nel subreddit per stilizzare il flair (es. "female", "sun", "blue")
'''
reddit_df['op_gender_visible'] = True
reddit_df['op_gender'] = ''
reddit_df.loc[reddit_df.author_flair_css_class.isin(['female']), 'op_gender'] = 'F'
reddit_df.loc[reddit_df.author_flair_css_class.isin(['male']), 'op_gender'] = 'M'

'''
Spiegazione della regex \d F|\dF|\/F

\d → matcha una cifra (es. 2, 5, 9)
\d F → trova pattern tipo "28 F"
\dF → trova pattern tipo "28F" (senza spazio)
\/F → trova /F come in "M/29"
'''
notmale = ~reddit_df.author_flair_text.str.contains('\d M|\dM|\/M', regex=True, na=False)
female = reddit_df.author_flair_text.str.contains('\d F|\dF|\/F', regex=True, na=False)
notfemale = ~reddit_df.author_flair_text.str.contains('\d F|\dF|\/F', regex=True, na=False)
male = reddit_df.author_flair_text.str.contains('\d M|\dM|\/M', regex=True, na=False)
notnull = reddit_df.author_flair_text.notnull()

reddit_df.loc[notnull & female & notmale, 'op_gender'] = 'F'
reddit_df.loc[notnull & male & notfemale, 'op_gender'] = 'M'
# se il flair non è né F né M, lo lasciamo vuoto
reddit_df = reddit_df[reddit_df.op_gender != '']

In [28]:
reddit_df.head(2)

,gender,body,author,author_flair_text,created_utc,link_id,score,controversiality,gilded,id,subreddit,author_flair_css_class,parent_id,reply_text,sub_id,op_gender_visible,op_gender
0,female,if this isn’t accurate then idk what is,be-concerned,15F,1542407736,t3_9xpaoc,60,0,0,e9ufsur,infj,sun,e9ufsur,"Yeah, this is definitely me. Then again, I cou...",12,True,F
1,female,"i’m in highschool right now. two years ago, my...",be-concerned,15F,1545673571,t3_a940fq,5,0,0,ecgumma,infj,sun,ecgumma,Identify a common interest you have and maybe ...,12,True,F


In [29]:
reddit_df.columns

Index(['gender', 'body', 'author', 'author_flair_text', 'created_utc',
       'link_id', 'score', 'controversiality', 'gilded', 'id', 'subreddit',
       'author_flair_css_class', 'parent_id', 'reply_text', 'sub_id',
       'op_gender_visible', 'op_gender'],
      dtype='object')